## Homogeneous Gas

Here is a notebook for homogeneous gas model.

Here we are talking about a homogeneous gas bulk of neutrinos with single energy. The EoM is
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\rho_E \right]
$$

while the EoM for antineutrinos is
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F \int_0^\infty dE' ( \rho_{E'} - \bar \rho_{E'} ) ,\bar\rho_E \right]
$$

Initial:
Homogeneous, Isotropic, Monoenergetic $\nu_e$ and $\bar\nu_e$

The equations becomes
$$
i \partial_t \rho_E = \left[ \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F   ( \rho_{E} - \bar \rho_{E} ) ,\rho_E \right]
$$
$$
i \partial_t \bar\rho_E = \left[- \frac{\delta m^2}{2E}B +\lambda L +\sqrt 2 G_F ( \rho_{E} - \bar \rho_{E} ) ,\bar\rho_E \right]
$$

where

$$
B = \frac{1}{2} \begin{pmatrix} 
-\cos 2\theta_v & \sin 2\theta_v \\
\sin 2\theta_v & \cos 2\theta_v
\end{pmatrix} = 
\begin{pmatrix} 
-0.38729833462 & 0.31622776601\\
0.31622776601 & 0.38729833462
\end{pmatrix}
$$

$$
L = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

Initial condition 
$$
\rho(t=0) = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

$$
\bar\rho(t=0) =\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

define the following quantities

1. hbar$=\hbar$
2. delm2E$= \delta m^2/2E$
3. lamb $= \lambda$
4. gF $= G_F$

## Numerical

In [64]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline
%load_ext snakeviz

import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D

import timeit

import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [65]:
hbar=1
delm2E=1
lamb=1
gF=1

## Here are some matrices to be used

elM = np.array([[1,0],[0,0]])
bM = 1/2*np.array( [ [ - 0.38729833462,0.31622776601] , [0.31622776601,0.38729833462] ] )

## sqareroot of 2
sqrt2=np.sqrt(2)

Using Mathematica, I can find the 4\*2 equations

In [66]:

#r11prime(t)

## The matrix eqn for neutrinos. Symplify the equation to the form A.X=0. Here I am only writing down the LHS.
## Eqn for r11'
# 1/2*( r21(t)*( bM12*delm2E - 2*sqrt2*gF*rb12(t) ) + r12(t) * ( -bM21*delm2E + 2*sqrt2*gF*rb21(t) ) - 1j*r11prime(t)  )
## Eqn for r12'
# 1/2*( r22(t)* ( bM12 ) )

### wait a minute I don't actually need to write down this. I can just do this part in numpy.

I am going to substitute all density matrix elements using their corrosponding network expressions.

So first of all, I need the network expression for the unknown functions.

A function is written as

$$ y_i= 1+t_i v_k f(t_i w_k+u_k) ,$$

while it's derivative is

$$v_k f(t w_k+u_k) + t v_k f(tw_k+u_k) (1-f(tw_k+u_k)) w_k .$$

Now I can write down the equations using these two forms.

In [67]:
def trigf(x):
    #return 1/(1+np.exp(-x)) # It's not bad to define this function here for people could use other functions other than expit(x).
    return expit(x)

In [68]:
## The time derivative part

### Here are the initial conditions

init = np.array( [[1,0],[0,1]] )
initb = np.array([[0,0],[0,0]])

### For neutrinos

def rho(x,ti,initialCondition): # x is the input structure arrays, ti is a time point

    v11,w11,u11,v12,w12,u12,v21,w21,u21,v22,w22,u22 = np.split(x,12)[:12]
        
    elem11= np.sum(ti * v11 * trigf( ti*w11 +u11 ) )
    elem12= np.sum(ti * v12 * trigf( ti*w12 +u12 ) )
    elem21= np.sum(ti * v21 * trigf( ti*w21 +u21 ) )
    elem22= np.sum(ti * v22 * trigf( ti*w22 +u22 ) )
    
    return initialCondition + np.array([[ elem11 , elem12 ],[elem21, elem22]])

def rhob(xb,ti,initialConditionb): # x is the input structure arrays, ti is a time point

    vb11,wb11,ub11,vb12,wb12,ub12,vb21,wb21,ub21,vb22,wb22,ub22 = np.split(xb,12)[:12]

    elem11= np.sum(ti * vb11 * trigf( ti*wb11 +ub11 ) )
    elem12= np.sum(ti * vb12 * trigf( ti*wb12 +ub12 ) )
    elem21= np.sum(ti * vb21 * trigf( ti*wb21 +ub21 ) )
    elem22= np.sum(ti * vb22 * trigf( ti*wb22 +ub22 ) )
    
    return initialConditionb + np.array([[ elem11 , elem12 ],[elem21, elem22]])



In [69]:
## Test
xtemp=np.ones(120)
rho(xtemp,1,init)

array([[ 9.80797078,  8.80797078],
       [ 8.80797078,  9.80797078]])

In [70]:
## Define Hamiltonians for both

def hamil(x,xb,ti,initialCondition,initialConditionb):
    
    return delm2E*bM + lamb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )


def hamilb(x,xb,ti,initialCondition,initialConditionb):
    
    return -delm2E*bM + lamb*elM + sqrt2*gF*( rho(x,ti,initialCondition) - rhob(xb,ti,initialConditionb) )

In [71]:
## The commutator

def comm(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamil(x,xb,ti,initialCondition,initialConditionb), rho(x,ti,initialCondition) ) - np.dot(rho(x,ti,initialCondition), hamil(x,xb,ti,initialCondition,initialConditionb) )

def commb(x,xb,ti,initialCondition,initialConditionb):
    
    return np.dot(hamilb(x,xb,ti,initialCondition,initialConditionb), rhob(xb,ti,initialConditionb) ) - np.dot(rhob(xb,ti,initialConditionb), hamilb(x,xb,ti,initialCondition,initialConditionb) )


In [81]:
## Test

print "neutrino\n",comm(xtemp,xtemp,1,init,initb)
print "antineutrino\n",commb(xtemp,xtemp,0.5,init,initb)

neutrino
[[ 0.          8.80797078]
 [-8.80797078  0.        ]]
antineutrino
[[ 0.          4.08787238]
 [-4.08787238  0.        ]]


In [73]:
## The COST of the eqn set

def costi(x,xb,ti,initialCondition):
    
    fvec11 = np.array(trigf(ti*w11 + u11) )  # This is a vector!!!
    
    costi11=
    costi12=

SyntaxError: invalid syntax (<ipython-input-73-e79803d1bf4c>, line 7)

In [ ]:
xtemp1 = np.arange(4)
xtemp1.shape = (2,2)
print xtemp1
xtemp1[0,1]
np.dot(xtemp1,xtemp1)